# pycisTopic analysis

Cell downsampled dataset, using consensus peak regions.

In [1]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [3]:
import pickle
import pandas as pd

In [4]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211027_hca_benchmark_full_merged//'
os.chdir( wdir )

In [5]:
import glob
from collections import OrderedDict
filenames = glob.glob('fragments_postbap/*.sinto.mm.fragments.tsv.gz')
samples = [item.replace(".sinto.mm.fragments.tsv.gz", "") for item in filenames]
samples = [item.replace("fragments_postbap/", "") for item in samples]
fragments_dict = {samples[i]: filenames[i] for i in range(len(samples))}
fragments_dict = OrderedDict(sorted(fragments_dict.items()))
fragments_dict.keys()

odict_keys(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'])

In [6]:
fragments_dict

OrderedDict([('Broad_1',
              'fragments_postbap/Broad_1.sinto.mm.fragments.tsv.gz'),
             ('Broad_2',
              'fragments_postbap/Broad_2.sinto.mm.fragments.tsv.gz'),
             ('Broad_mito_1',
              'fragments_postbap/Broad_mito_1.sinto.mm.fragments.tsv.gz'),
             ('Broad_mito_2',
              'fragments_postbap/Broad_mito_2.sinto.mm.fragments.tsv.gz'),
             ('CNAG_1', 'fragments_postbap/CNAG_1.sinto.mm.fragments.tsv.gz'),
             ('CNAG_2', 'fragments_postbap/CNAG_2.sinto.mm.fragments.tsv.gz'),
             ('Sanger_1',
              'fragments_postbap/Sanger_1.sinto.mm.fragments.tsv.gz'),
             ('Sanger_2',
              'fragments_postbap/Sanger_2.sinto.mm.fragments.tsv.gz'),
             ('Stanford_1',
              'fragments_postbap/Stanford_1.sinto.mm.fragments.tsv.gz'),
             ('Stanford_2',
              'fragments_postbap/Stanford_2.sinto.mm.fragments.tsv.gz'),
             ('VIB_1', 'fragments_postbap/VIB_

# load consensus peak dict


load a dictionary with paths to consensus files, generated in notebook 3

In [7]:
import glob
from collections import OrderedDict
consensus_peaks_dict = {}
key = 'full_merged'
consensus_regions = 'pycistopic_screen/consensus_peak_calling/'+key+'/'+key+'__consensus_regions.bed'

In [8]:
filenames = glob.glob('fragments_postbap/*.sinto.mm.fragments.tsv.gz')
samples = [item.replace(".sinto.mm.fragments.tsv.gz", "") for item in filenames]
samples = [item.replace("fragments_postbap/", "") for item in samples]
consensus_peaks_dict = {samples[i]: consensus_regions for i in range(len(samples))}
consensus_peaks_dict = OrderedDict(sorted(consensus_peaks_dict.items()))
consensus_peaks_dict.keys()

odict_keys(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'])

## Library QC metrics

In [9]:
from pycisTopic.qc import compute_qc_stats
import pyranges

In [10]:
!cp /staging/leuven/stg_00002/lcb/cflerin/analysis/pbmc_atac/analysis2/data_freeze_Jun2021/atac_qc_multiplet_merged/jupyter/biomart_annot.pickle .

In [11]:
import pickle
f_biomart = 'biomart_annot.pickle'
with open(f_biomart, 'rb') as f:
    annot = pickle.load(f)
annot

,Chromosome,Start,Strand,Gene,Transcript_type
8986,chrY,9337464,1,TSPY4,protein_coding
8987,chrY,9337510,1,TSPY4,protein_coding
9024,chrY,22490397,1,PRY,protein_coding
9100,chrY,14056227,1,VCY1B,protein_coding
9136,chrY,12662368,1,USP9Y,protein_coding
...,...,...,...,...,...
236024,chr1,36479519,-1,CSF3R,protein_coding
236025,chr1,36471474,-1,CSF3R,protein_coding
236033,chr1,36482051,-1,CSF3R,protein_coding
236034,chr1,36323645,-1,EVA1B,protein_coding


In [12]:
import ray
ray.shutdown()

metadata_bc_dict, profile_data_dict = compute_qc_stats(
        fragments_dict=fragments_dict,
        tss_annotation=annot,
        stats=['barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'frip'],
        label_list=None,
        path_to_regions=consensus_peaks_dict,
        n_cpu=24,
        valid_bc=None,
        n_frag=100,
        n_bc=None,
        tss_flank_window=2000,
        tss_window=50,
        tss_minimum_signal_window=100,
        tss_rolling_window=10,
        min_norm=0.1,
        remove_duplicates = True,
        )

2021-11-27 14:15:46,509 cisTopic     INFO     n_cpu is larger than the number of samples. Setting n_cpu to the number of samples


2021-11-27 14:15:48,394	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=33421) 2021-11-27 14:15:52,959 cisTopic     INFO     Reading Broad_mito_1
(pid=33420) 2021-11-27 14:15:53,204 cisTopic     INFO     Reading Sanger_1
(pid=33419) 2021-11-27 14:15:53,143 cisTopic     INFO     Reading Broad_mito_2
(pid=33426) 2021-11-27 14:15:53,155 cisTopic     INFO     Reading Sanger_2
(pid=33425) 2021-11-27 14:15:53,203 cisTopic     INFO     Reading Broad_1
(pid=33424) 2021-11-27 14:15:53,150 cisTopic     INFO     Reading Stanford_1
(pid=33430) 2021-11-27 14:15:53,192 cisTopic     INFO     Reading CNAG_1
(pid=33431) 2021-11-27 14:15:53,134 cisTopic     INFO     Reading Stanford_2
(pid=33423) 2021-11-27 14:15:53,210 cisTopic     INFO     Reading VIB_Hydrop_2
(pid=33428) 2021-11-27 14:15:53,211 cisTopic     INFO     Reading Broad_2
(pid=33418) 2021-11-27 14:15:53,245 cisTopic     INFO     Reading s3atac
(pid=33429) 2021-11-27 14:15:53,257 cisTopic     INFO     Reading VIB_1
(pid=33427) 2021-11-27 14:15:53,259 cisTopic     INFO     Reading VIB_Hydrop_1
(pid=33422) 20

In [13]:
f_qc_dir = 'pycistopic_consensus/qc__consensus_peaks'
if not os.path.exists(os.path.join(wdir, f_qc_dir)):
    os.makedirs(os.path.join(wdir, f_qc_dir))

In [14]:
with open(os.path.join(wdir, f_qc_dir, 'metadata.pickle'), 'wb') as f:
    pickle.dump(metadata_bc_dict, f)

with open(os.path.join(wdir, f_qc_dir, 'profile_data.pickle'), 'wb') as f:
    pickle.dump(profile_data_dict, f)